In [1]:
#Librerias que se utilizarán
# import pandas as pd
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, make_scorer
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
import itertools
from joblib import Parallel, delayed
import multiprocessing


import pandas as pd
from GMHI_functions import *







path=pd.read_csv('../../DataSets/CAMDA_2025/pathways_article.zip',sep=',')

In [2]:
path['path']=path['Unnamed: 0'].str.replace(' ','_')
path.drop(columns='Unnamed: 0',inplace=True)
path=path.loc[path['path'][path['path'].str.contains(r'\|')].index,]
unintegrated_index=path['path'][path['path'].str.contains('UNINTEGRATED')].values
integrated_index=list(set(path['path']).difference(set(unintegrated_index)))
path.set_index('path',inplace=True)
UNINTEGRATED=path.loc[unintegrated_index,]
INTEGRATED=path.loc[integrated_index,]

In [3]:
tax=pd.read_csv('../../DataSets/CAMDA_2025/tax_article_dep.csv',sep=',',index_col=0)

In [4]:
meta=pd.read_csv('../../DataSets/CAMDA_2025/metadata_article.csv',sep=',')
disease_col_name='disease'
sample_name='sample'
diagnosis_col_name='disease_group'


meta[diagnosis_col_name]=pd.Series(index=meta.index,dtype=str)


meta.loc[meta[meta[disease_col_name].isin(["T2D","IGT","metabolic_syndrome"])].index,diagnosis_col_name]='Metabolic Conditions'
meta.loc[meta[meta[disease_col_name].isin(['ACVD',"CAD"])].index,diagnosis_col_name]='Cardiovascular Diseases'
meta.loc[meta[meta[disease_col_name].isin(["PD","ME/CFS"])].index,diagnosis_col_name]='Neurological Disorders'
meta.loc[meta[meta[disease_col_name].isin(["BD","schizophrenia"])].index,diagnosis_col_name]='Psychiatric Disorders'
meta.loc[meta[meta[disease_col_name].isin(['healthy'])].index,diagnosis_col_name]='Healthy'
meta.loc[meta[meta[disease_col_name].isin(["acute_diarrhoea","CRC","adenoma","IBD","CD","UC","cirrhosis","STH","few_polyps"])].index,diagnosis_col_name]='Gastrointestinal diseases'
meta.drop(index=meta[meta['disease_group'].isna()].index,inplace=True)

path=path[meta[sample_name]]
UNINTEGRATED=UNINTEGRATED[meta[sample_name]]
INTEGRATED=INTEGRATED[meta[sample_name]]
tax=tax[meta[sample_name]]

In [5]:
def get_combinations(elements,places):
    return list(itertools.combinations(elements,places))



def restrict_meta_to_first_and_second(meta,diagnosis_col_name,first_group_conditions,second_group_conditions):
    restricted_meta=meta.copy()
    new_first_group_conditions=' and '.join(first_group_conditions)
    new_second_group_conditions=' and '.join(second_group_conditions)
    restricted_meta.loc[restricted_meta[restricted_meta[diagnosis_col_name].isin(first_group_conditions)].index,diagnosis_col_name] = new_first_group_conditions
    restricted_meta.loc[restricted_meta[restricted_meta[diagnosis_col_name].isin(second_group_conditions)].index,diagnosis_col_name] = new_second_group_conditions
    return restricted_meta,new_first_group_conditions,new_second_group_conditions








def train_RF_GMHI(df_vs_metrics, diagnosis_col_name, test_size=0.2, random_state=42):
    # Separar X e y
    X = df_vs_metrics.drop(columns=[diagnosis_col_name])
    y = df_vs_metrics[diagnosis_col_name]

    # División train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    # Detectar clases (binario)
    clases = sorted(y.unique())
    positive_class = clases[1]

    # Definir rango de hiperparámetros
    param_grid = {
        'n_estimators': [100, 200,300,400],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'bootstrap': [True, False]
    }

    # Obtener todas las combinaciones posibles
    keys, values = zip(*param_grid.items())
    all_param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # Buscar el mejor modelo basado en F1 en test
    best_f1 = -1
    best_acc = None
    best_params = None

    for params in all_param_combinations:
        model = RandomForestClassifier(**params, random_state=random_state)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        f1 = f1_score(y_test, y_pred, pos_label=positive_class)
        acc = accuracy_score(y_test, y_pred)

        if f1 > best_f1:
            best_f1 = f1
            best_acc = acc
            best_params = params

    return best_acc, best_f1, best_params






In [23]:
def gmhi_tree(meta,diagnosis_col_name,sample_name,UNINTEGRATED,INTEGRATED,tax,df_best_vs=None,first_execution=True):
    
    all_diagnosis=set(meta[diagnosis_col_name].unique())
    

    if len(all_diagnosis)==1:
        return df_best_vs
    
    combinations=get_combinations(all_diagnosis,len(all_diagnosis)//2)
    

    df_comparative_vs=pd.DataFrame(columns=['mean_accuracy','mean_f1','sets_UNINTEGRATED_f1','sets_INTEGRATED_f1','sets_tax_f1'])
    

    if first_execution == True:
        df_best_vs=pd.DataFrame(columns=['mean_accuracy','mean_f1','sets_UNINTEGRATED_f1','sets_INTEGRATED_f1','sets_tax_f1','RF_accuracy','RF_f1','RF_params_f1'])

    dict_GMHI_vs={}

    for c in combinations:
        df_for_RF=pd.DataFrame(index=meta[sample_name].values)
        

        first_group_conditions=list(c)
        second_group_conditions=list(all_diagnosis.difference(c))
        restricted_meta,first_group_conditions_name,second_group_conditions_name=restrict_meta_to_first_and_second(meta,diagnosis_col_name,first_group_conditions,second_group_conditions)
        vs_name=first_group_conditions_name+' vs '+second_group_conditions_name
        

        
        UNINTEGRATED_combination=gmhi_pairwise_gridsearch(restricted_meta, UNINTEGRATED, sample_name, diagnosis_col_name,p1_lower=0.1,p2_lower=0.01,p3_lower=0.1,p4_lower=0.01,p1_upper=2.5,p2_upper=0.5,p3_upper=2.5,p4_upper=0.5,grid_len=9,only_pair=True)
        INTEGRATED_combination=gmhi_pairwise_gridsearch(restricted_meta, INTEGRATED, sample_name, diagnosis_col_name,p1_lower=0.1,p2_lower=0.01,p3_lower=0.1,p4_lower=0.01,p1_upper=2.5,p2_upper=0.5,p3_upper=2.5,p4_upper=0.5,grid_len=9,only_pair=True)
        tax_combination=gmhi_pairwise_gridsearch(restricted_meta, tax, sample_name, diagnosis_col_name,p1_lower=0.1,p2_lower=0.01,p3_lower=0.1,p4_lower=0.01,p1_upper=2.5,p2_upper=0.5,p3_upper=2.5,p4_upper=0.5,grid_len=9,only_pair=True)

        ##GMHI para RF
        
        df_for_RF['UNINTEGRATED']=UNINTEGRATED_combination[3]
        df_for_RF['INTEGRATED']=INTEGRATED_combination[3]
        df_for_RF['tax']=tax_combination[3]
        df_for_RF[diagnosis_col_name]=restricted_meta[[diagnosis_col_name,sample_name]].set_index(sample_name).copy()
        #Se guarda en un diccionario:
        dict_GMHI_vs[vs_name]=df_for_RF
        
        
        df_comparative_vs.loc[first_group_conditions_name+' vs '+second_group_conditions_name,]=[np.mean([UNINTEGRATED_combination[0],INTEGRATED_combination[0],tax_combination[0]]),
                                                                                       np.mean([UNINTEGRATED_combination[1],INTEGRATED_combination[1],tax_combination[1]]),
                                                                                       list(UNINTEGRATED_combination[5].items()),list(INTEGRATED_combination[5].items()),
                                                                                       list(tax_combination[5].items())]
    
    
    best_vs_name=df_comparative_vs['mean_f1'].idxmax()
    row=list(df_comparative_vs.loc[best_vs_name].values)
    
    row_2=train_RF_GMHI(dict_GMHI_vs[best_vs_name], diagnosis_col_name)
    
    row.extend(list(row_2))
    df_best_vs.loc[best_vs_name,]=row
    
    meta_first_group=meta[meta[diagnosis_col_name].isin(best_vs_name.split(' vs ')[0].split(' and '))].copy()
    UNINTEGRATED_first_group=UNINTEGRATED[meta_first_group[sample_name]].copy()
    INTEGRATED_first_group=INTEGRATED[meta_first_group[sample_name]].copy()
    tax_first_group=tax[meta_first_group[sample_name]].copy()
    
    meta_second_group=meta[meta[diagnosis_col_name].isin(best_vs_name.split(' vs ')[1].split(' and '))].copy()
    UNINTEGRATED_second_group=UNINTEGRATED[meta_second_group[sample_name]].copy()
    INTEGRATED_second_group=INTEGRATED[meta_second_group[sample_name]].copy()
    tax_second_group=tax[meta_second_group[sample_name]].copy()


    df_best_vs=gmhi_tree(meta_first_group,diagnosis_col_name,sample_name,UNINTEGRATED_first_group,INTEGRATED_first_group,tax_first_group,df_best_vs=df_best_vs,first_execution=False)
    df_best_vs=gmhi_tree(meta_second_group,diagnosis_col_name,sample_name,UNINTEGRATED_second_group,INTEGRATED_second_group,tax_second_group,df_best_vs=df_best_vs,first_execution=False)
    
    return df_best_vs

In [24]:
a=gmhi_tree(meta,diagnosis_col_name,sample_name,UNINTEGRATED,INTEGRATED,tax)

Los mejores parámetros para Healthy and Metabolic Conditions and Neurological Disorders vs Cardiovascular Diseases and Gastrointestinal diseases and Psychiatric Disorders son accuracy: 0.1,0.07125,0.1,0.01 | f1: 1.9,0.01,1.9,0.01
Los mejores parámetros para Healthy and Metabolic Conditions and Neurological Disorders vs Cardiovascular Diseases and Gastrointestinal diseases and Psychiatric Disorders son accuracy: 0.1,0.01,1.6,0.19375 | f1: 0.1,0.1325,0.1,0.19375
Los mejores parámetros para Healthy and Metabolic Conditions and Neurological Disorders vs Cardiovascular Diseases and Gastrointestinal diseases and Psychiatric Disorders son accuracy: 0.1,0.07125,1.3,0.01 | f1: 1.3,0.07125,0.1,0.07125
Los mejores parámetros para Healthy and Metabolic Conditions and Psychiatric Disorders vs Cardiovascular Diseases and Neurological Disorders and Gastrointestinal diseases son accuracy: 0.1,0.07125,0.4,0.01 | f1: 1.9,0.01,1.9,0.01
Los mejores parámetros para Healthy and Metabolic Conditions and Psyc

In [27]:
a.to_csv('BUENOS_PARAM.csv')

In [29]:
a

,mean_accuracy,mean_f1,sets_UNINTEGRATED_f1,sets_INTEGRATED_f1,sets_tax_f1,RF_accuracy,RF_f1,RF_params_f1
Healthy and Metabolic Conditions and Gastrointestinal diseases vs Cardiovascular Diseases and Neurological Disorders and Psychiatric Disorders,0.799405,0.683277,[(Healthy and Metabolic Conditions and Gastroi...,[(Healthy and Metabolic Conditions and Gastroi...,[(Healthy and Metabolic Conditions and Gastroi...,0.878981,0.935264,"{'n_estimators': 300, 'max_depth': 10, 'min_sa..."
Gastrointestinal diseases vs Healthy and Metabolic Conditions,0.729264,0.684198,"[(Healthy and Metabolic Conditions, {UNINTEGRA...","[(Healthy and Metabolic Conditions, {PWY-5304:...","[(Healthy and Metabolic Conditions, {Prevotell...",0.794224,0.858209,"{'n_estimators': 200, 'max_depth': None, 'min_..."
Healthy vs Metabolic Conditions,0.71414,0.646637,"[(Healthy, {UNINTEGRATED|g__Bacteroides.s__Bac...","[(Healthy, {PWY-7237:_myo-,_chiro-_and_scillo-...","[(Healthy, {Weissella_confusa, Citrobacter_kos...",0.760204,0.560748,"{'n_estimators': 100, 'max_depth': None, 'min_..."
Neurological Disorders vs Cardiovascular Diseases and Psychiatric Disorders,0.868468,0.870534,"[(Neurological Disorders, {UNINTEGRATED|g__Clo...","[(Neurological Disorders, {VALSYN-PWY:_L-valin...","[(Neurological Disorders, {Bifidobacterium_ani...",0.972973,0.916667,"{'n_estimators': 100, 'max_depth': None, 'min_..."
Cardiovascular Diseases vs Psychiatric Disorders,0.691803,0.847608,"[(Cardiovascular Diseases, {UNINTEGRATED|g__St...","[(Cardiovascular Diseases, {PWY-6121:_5-aminoi...","[(Cardiovascular Diseases, {Candidatus_Stoquef...",0.918033,0.761905,"{'n_estimators': 100, 'max_depth': None, 'min_..."


In [36]:
gmhi_tree_models_metrics=pd.read_csv('BUENOS_PARAM.csv',index_col=0)
gmhi_tree_models_metrics

,mean_accuracy,mean_f1,sets_UNINTEGRATED_f1,sets_INTEGRATED_f1,sets_tax_f1,RF_accuracy,RF_f1,RF_params_f1
Healthy and Metabolic Conditions and Gastrointestinal diseases vs Cardiovascular Diseases and Neurological Disorders and Psychiatric Disorders,0.799405,0.683277,[('Healthy and Metabolic Conditions and Gastro...,[('Healthy and Metabolic Conditions and Gastro...,[('Healthy and Metabolic Conditions and Gastro...,0.878981,0.935264,"{'n_estimators': 300, 'max_depth': 10, 'min_sa..."
Gastrointestinal diseases vs Healthy and Metabolic Conditions,0.729264,0.684198,"[('Healthy and Metabolic Conditions', {'UNINTE...","[('Healthy and Metabolic Conditions', {'PWY-53...","[('Healthy and Metabolic Conditions', {'Prevot...",0.794224,0.858209,"{'n_estimators': 200, 'max_depth': None, 'min_..."
Healthy vs Metabolic Conditions,0.714140,0.646637,"[('Healthy', {'UNINTEGRATED|g__Bacteroides.s__...","[('Healthy', {'PWY-7237:_myo-,_chiro-_and_scil...","[('Healthy', {'Weissella_confusa', 'Citrobacte...",0.760204,0.560748,"{'n_estimators': 100, 'max_depth': None, 'min_..."
Neurological Disorders vs Cardiovascular Diseases and Psychiatric Disorders,0.868468,0.870534,"[('Neurological Disorders', {'UNINTEGRATED|g__...","[('Neurological Disorders', {'VALSYN-PWY:_L-va...","[('Neurological Disorders', {'Bifidobacterium_...",0.972973,0.916667,"{'n_estimators': 100, 'max_depth': None, 'min_..."
Cardiovascular Diseases vs Psychiatric Disorders,0.691803,0.847608,"[('Cardiovascular Diseases', {'UNINTEGRATED|g_...","[('Cardiovascular Diseases', {'PWY-6121:_5-ami...","[('Cardiovascular Diseases', {'Candidatus_Stoq...",0.918033,0.761905,"{'n_estimators': 100, 'max_depth': None, 'min_..."


In [69]:
def get_gmhi_from_gmhi_tree(gmhi_tree_models_metrics,vs,tables):
    columns=['sets_UNINTEGRATED_f1','sets_INTEGRATED_f1','sets_tax_f1']
    columns_and_tables=list(zip(columns,tables))

    three_gmhis=[]
    for c,t in columns_and_tables:
        MH,MN=gmhi_tree_models_metrics.loc[vs,c]
        three_gmhis.append(get_all_GMHI(t,MH,MN))
    return three_gmhis

In [70]:
get_gmhi_from_gmhi_tree(gmhi_tree_models_metrics,'Healthy and Metabolic Conditions and Gastrointestinal diseases vs Cardiovascular Diseases and Neurological Disorders and Psychiatric Disorders',[UNINTEGRATED,INTEGRATED,tax])

ValueError: too many values to unpack (expected 2)

In [79]:
gmhi_tree_models_metrics.loc['Healthy and Metabolic Conditions and Gastrointestinal diseases vs Cardiovascular Diseases and Neurological Disorders and Psychiatric Disorders','sets_UNINTEGRATED_f1']

"[('Healthy and Metabolic Conditions and Gastrointestinal diseases', {'UNINTEGRATED|g__Bacteroides.s__Bacteroides_sp_CAG_144', 'UNINTEGRATED|g__Methanobrevibacter.s__Methanobrevibacter_smithii', 'UNINTEGRATED|g__Barnesiella.s__Barnesiella_intestinihominis', 'UNINTEGRATED|g__Lawsonibacter.s__Lawsonibacter_asaccharolyticus', 'UNINTEGRATED|g__Phascolarctobacterium.s__Phascolarctobacterium_succinatutens', 'UNINTEGRATED|g__Methanobrevibacter.s__Methanobrevibacter_smithii_CAG_186', 'UNINTEGRATED|g__Streptococcus.s__Streptococcus_thermophilus_CAG_236', 'UNINTEGRATED|g__Streptococcus.s__Streptococcus_thermophilus'}), ('Cardiovascular Diseases and Neurological Disorders and Psychiatric Disorders', {'UNINTEGRATED|g__Lactobacillus.s__Lactobacillus_salivarius', 'UNINTEGRATED|g__Dialister.s__Dialister_sp_CAG_357', 'UNINTEGRATED|g__Megasphaera.s__Megasphaera_elsdenii', 'UNINTEGRATED|g__Eisenbergiella.s__Eisenbergiella_tayi', 'UNINTEGRATED|g__Lachnoclostridium.s__Clostridium_asparagiforme', 'UNINTEGR

In [84]:
meta['disease_group'].unique()

array(['Healthy', 'Neurological Disorders', 'Gastrointestinal diseases',
       'Metabolic Conditions', 'Cardiovascular Diseases',
       'Psychiatric Disorders'], dtype=object)

In [81]:
import ast

s = gmhi_tree_models_metrics
lista = ast.literal_eval(s)

ValueError: malformed node or string:                                                     mean_accuracy   mean_f1  \
Healthy and Metabolic Conditions and Gastrointe...       0.799405  0.683277   
Gastrointestinal diseases vs Healthy and Metabo...       0.729264  0.684198   
Healthy vs Metabolic Conditions                          0.714140  0.646637   
Neurological Disorders vs Cardiovascular Diseas...       0.868468  0.870534   
Cardiovascular Diseases vs Psychiatric Disorders         0.691803  0.847608   

                                                                                 sets_UNINTEGRATED_f1  \
Healthy and Metabolic Conditions and Gastrointe...  [('Healthy and Metabolic Conditions and Gastro...   
Gastrointestinal diseases vs Healthy and Metabo...  [('Healthy and Metabolic Conditions', {'UNINTE...   
Healthy vs Metabolic Conditions                     [('Healthy', {'UNINTEGRATED|g__Bacteroides.s__...   
Neurological Disorders vs Cardiovascular Diseas...  [('Neurological

In [64]:
b=[('sets_UNINTEGRATED_f1', 1), ('sets_INTEGRATED_f1', 2), ('sets_tax_f1', 3)]
for i,j in b:
    print(i)

sets_UNINTEGRATED_f1
sets_INTEGRATED_f1
sets_tax_f1


In [35]:
from sklearn.ensemble import RandomForestClassifier

def get_gmhi_tree_models(gmhi_tree_models_metrics, meta,UNINTEGRATED,INTEGRATED,tax):
    models = {}
    for vs in gmhi_tree_models_metrics.index:
        MH,MN=gmhi_tree_models_metrics.loc[vs, 'RF_params_f1']

        rf_params = gmhi_tree_models_metrics.loc[vs, 'RF_params_f1']
        model = RandomForestClassifier(**rf_params)
        models[vs] = model
    return models


In [ ]:
def train_RF_GMHI(df_vs_metrics, diagnosis_col_name, test_size=0.2, random_state=42):
    # Separar X e y
    X = df_vs_metrics.drop(columns=[diagnosis_col_name])
    y = df_vs_metrics[diagnosis_col_name]

    # División train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    # Detectar clases (binario)
    clases = sorted(y.unique())
    positive_class = clases[1]

    # Definir rango de hiperparámetros
    param_grid = {
        'n_estimators': [100, 200,300,400],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'bootstrap': [True, False]
    }

    # Obtener todas las combinaciones posibles
    keys, values = zip(*param_grid.items())
    all_param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # Buscar el mejor modelo basado en F1 en test
    best_f1 = -1
    best_acc = None
    best_params = None

    for params in all_param_combinations:
        model = RandomForestClassifier(**params, random_state=random_state)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        f1 = f1_score(y_test, y_pred, pos_label=positive_class)
        acc = accuracy_score(y_test, y_pred)

        if f1 > best_f1:
            best_f1 = f1
            best_acc = acc
            best_params = params

    return best_acc, best_f1, best_params

In [34]:
get_gmhi_tree_models(gmhi_tree_models_metrics, tax)

NameError: name 'models' is not defined

In [ ]:
b=gmhi_tree(meta,diagnosis_col_name,sample_name,UNINTEGRATED,INTEGRATED,tax)

Los mejores parámetros para Healthy and Neurological Disorders and Cardiovascular Diseases vs Metabolic Conditions and Psychiatric Disorders and Gastrointestinal diseases son accuracy: 1.3,0.01,0.1,0.01 | f1: 1.3,0.01,0.1,0.01


In [ ]:
def model_per_param(params):
    p1,p2,p3,p4=params[2:6]
    p1_p2=str(p1)+','+str(p2)
    p3_p4=str(p3)+','+str(p4)
    index_name=p1_p2+','+p3_p4
    
    GMHI,MH,MN,metrics=train_GMHI(params[0],params[1],p1,p2,p3,p4,params[6],params[7],params[8])
    
    p1_p2=str(p1)+','+str(p2)
    p3_p4=str(p3)+','+str(p4)
    row=[p1_p2,p3_p4]
    
    row.extend([metrics[0],metrics[1],[MH,MN,GMHI]])
    

    return p1_p2+','+p3_p4, row

def gmhi_pairwise_gridsearch(meta, tax, sample_name, diagnosis_col_name,p1_lower=0,p2_lower=0,p3_lower=0,p4_lower=0,p1_upper=0.7,p2_upper=0.7,p3_upper=0.7,p4_upper=0.7,grid_len=7,only_pair=False):
    all_diagnosis=meta[diagnosis_col_name].unique()
    cols_and_index = np.append(all_diagnosis, 'All')
    df_accuracy = pd.DataFrame(index=cols_and_index, columns=cols_and_index, data=np.nan)
    df_f1 = pd.DataFrame(index=cols_and_index, columns=cols_and_index, data=np.nan)

    df_GMHI_accuracy = pd.DataFrame(index=tax.columns)
    df_GMHI_f1 = pd.DataFrame(index=tax.columns)

    dict_best_sets_accuracy = {}
    dict_best_sets_f1 = {}

    list_pairs = list(itertools.combinations(cols_and_index, 2))

    for vs in list_pairs:
        first_condition=vs[0]
        second_condition=vs[1]
        
        vs_name=first_condition+' vs '+ second_condition
        if only_pair==True and 'All' in vs_name:
            continue
        
        meta_first_and_second_condition=meta[meta[diagnosis_col_name].isin(vs)].copy()
        tax_first_and_second_condition=tax[meta_first_and_second_condition[sample_name]].copy()
        df_metrics = pd.DataFrame(columns=['p1,p2', 'p3,p4', 'accuracy', 'f1', 'MH_MN_GMHI'])

        param_grid = [[meta_first_and_second_condition, tax_first_and_second_condition, p1, p2, p3, p4,
                           sample_name, first_condition, diagnosis_col_name]
                          for p1 in np.linspace(p1_lower, p1_upper, num=grid_len)
                          for p2 in np.linspace(p2_lower, p2_upper, num=grid_len)
                          for p3 in np.linspace(p3_lower, p3_upper, num=grid_len)
                          for p4 in np.linspace(p4_lower, p4_upper, num=grid_len)]
        for params in param_grid:
            row=model_per_param(params)
            df_metrics.loc[row[0],]=row[1]

        best_param_for_accuracy = df_metrics['accuracy'].idxmax()
        best_param_for_f1 = df_metrics['f1'].idxmax()

        print(f'Los mejores parámetros para {vs_name} son '
                  f'accuracy: {best_param_for_accuracy} | f1: {best_param_for_f1}')

        best_GMHI_sets_for_accuracy = df_metrics.loc[best_param_for_accuracy, 'MH_MN_GMHI'][:2]
        best_GMHI_sets_for_f1 = df_metrics.loc[best_param_for_f1, 'MH_MN_GMHI'][:2]

        df_accuracy.loc[first_condition, second_condition] = df_metrics.loc[best_param_for_accuracy, 'accuracy']
        df_f1.loc[first_condition, second_condition] = df_metrics.loc[best_param_for_f1, 'f1']

        df_accuracy.loc[second_condition, first_condition] = df_accuracy.loc[first_condition, second_condition]
        df_f1.loc[second_condition, first_condition] = df_f1.loc[first_condition, second_condition]

        df_GMHI_accuracy[vs_name] = df_metrics.loc[best_param_for_f1, 'MH_MN_GMHI'][2]
        df_GMHI_f1[vs_name] = df_metrics.loc[best_param_for_accuracy, 'MH_MN_GMHI'][2]

        
        dict_best_sets_accuracy[vs_name]={first_condition:best_GMHI_sets_for_accuracy[0],second_condition:best_GMHI_sets_for_accuracy[1]}
        dict_best_sets_f1[vs_name]={first_condition:best_GMHI_sets_for_f1[0],second_condition:best_GMHI_sets_for_f1[1]}

        if first_condition=='Cardiovascular Diseases and Neurological Disorders':
            print(df_GMHI_accuracy[vs_name])
            
        if only_pair==True:
            return df_accuracy.loc[first_condition, second_condition],df_f1.loc[first_condition, second_condition],df_GMHI_accuracy[vs_name],df_GMHI_f1[vs_name],dict_best_sets_accuracy[vs_name],dict_best_sets_f1[vs_name]


    

    return df_accuracy, df_f1, df_GMHI_accuracy, df_GMHI_f1, dict_best_sets_accuracy, dict_best_sets_f1
    
        

    

In [89]:
meta_dsadas['a'].idxmax()

0

In [120]:
c=list(meta_dsadas.loc[0,['study_name','disease']].values)
d=meta_dsadas.loc[1,['study_name','disease']].values


In [121]:
c.extend(list(d))

In [122]:
c

[3, 4, 'BedarfJR_2017', 'healthy']

In [113]:
c


array([3, 4], dtype=object)

In [119]:
x

In [74]:
a=gmhi_pairwise_gridsearch(meta, UNINTEGRATED, sample_name, diagnosis_col_name,p1_lower=0,p2_lower=0,p3_lower=0,p4_lower=0,p1_upper=2,p2_upper=0.5,p3_upper=2,p4_upper=0.5,grid_len=1,only_pair=True)
a[0]

Los mejores parámetros para Healthy vs Neurological Disorders son accuracy: 0.0,0.0,0.0,0.0 | f1: 0.0,0.0,0.0,0.0


0.10437235543018336

In [75]:
a

(0.10437235543018336,
 0.532520325203252,
 MMPD37460080ST    -0.526941
 MMPD56131850ST    -0.988706
 MMPD19623701ST    -0.795355
 MMPD91893113ST    -0.932836
 MMPD45969642ST    -1.247850
                      ...   
 wHAXPI043592.8          NaN
 wHAXPI043593.9          NaN
 wHAXPI043594.11         NaN
 wHAXPI047830.11         NaN
 wHAXPI048670.90         NaN
 Name: Healthy vs Neurological Disorders, Length: 3139, dtype: float64,
 MMPD37460080ST    -0.526941
 MMPD56131850ST    -0.988706
 MMPD19623701ST    -0.795355
 MMPD91893113ST    -0.932836
 MMPD45969642ST    -1.247850
                      ...   
 wHAXPI043592.8          NaN
 wHAXPI043593.9          NaN
 wHAXPI043594.11         NaN
 wHAXPI047830.11         NaN
 wHAXPI048670.90         NaN
 Name: Healthy vs Neurological Disorders, Length: 3139, dtype: float64,
 {'Healthy': {'UNINTEGRATED|g__Acidaminococcus.s__Acidaminococcus_fermentans',
   'UNINTEGRATED|g__Acidaminococcus.s__Acidaminococcus_intestini',
   'UNINTEGRATED|g__Acidaminoc

{'Healthy': {'UNINTEGRATED|g__Acidaminococcus.s__Acidaminococcus_fermentans',
  'UNINTEGRATED|g__Acidaminococcus.s__Acidaminococcus_intestini',
  'UNINTEGRATED|g__Acidaminococcus.s__Acidaminococcus_intestini_CAG_325',
  'UNINTEGRATED|g__Aeromonas.s__Aeromonas_caviae',
  'UNINTEGRATED|g__Aeromonas.s__Aeromonas_veronii',
  'UNINTEGRATED|g__Agathobaculum.s__Agathobaculum_butyriciproducens',
  'UNINTEGRATED|g__Alistipes.s__Alistipes_indistinctus',
  'UNINTEGRATED|g__Alistipes.s__Alistipes_onderdonkii',
  'UNINTEGRATED|g__Alistipes.s__Alistipes_sp_An66',
  'UNINTEGRATED|g__Alistipes.s__Alistipes_sp_CAG_831',
  'UNINTEGRATED|g__Anaerococcus.s__Anaerococcus_vaginalis',
  'UNINTEGRATED|g__Anaeroglobus.s__Anaeroglobus_geminatus',
  'UNINTEGRATED|g__Anaerostipes.s__Anaerostipes_hadrus',
  'UNINTEGRATED|g__Bacteroides.s__Bacteroides_clarus',
  'UNINTEGRATED|g__Bacteroides.s__Bacteroides_coprocola',
  'UNINTEGRATED|g__Bacteroides.s__Bacteroides_coprocola_CAG_162',
  'UNINTEGRATED|g__Bacteroides.s_

In [14]:
combinations[0]

('Neurological Disorders', 'Gastrointestinal diseases', 'Metabolic Conditions')

In [ ]:
meta_combination=meta.copy()
meta_combination[]

In [6]:
a=gmhi_pairwise_gridsearch(meta, UNINTEGRATED, sample_name, diagnosis_col_name,p1_lower=0,p2_lower=0,p3_lower=0,p4_lower=0,p1_upper=2,p2_upper=0.5,p3_upper=2,p4_upper=0.5,grid_len=1)

Los mejores parámetros para Healthy vs All son accuracy: 0.0,0.0,0.0,0.0 | f1: 0.0,0.0,0.0,0.0
Los mejores parámetros para Healthy vs Neurological Disorders son accuracy: 0.0,0.0,0.0,0.0 | f1: 0.0,0.0,0.0,0.0
Los mejores parámetros para Healthy vs Gastrointestinal diseases son accuracy: 0.0,0.0,0.0,0.0 | f1: 0.0,0.0,0.0,0.0
Los mejores parámetros para Healthy vs Metabolic Conditions son accuracy: 0.0,0.0,0.0,0.0 | f1: 0.0,0.0,0.0,0.0
Los mejores parámetros para Healthy vs Cardiovascular Diseases son accuracy: 0.0,0.0,0.0,0.0 | f1: 0.0,0.0,0.0,0.0
Los mejores parámetros para Healthy vs Psychiatric Disorders son accuracy: 0.0,0.0,0.0,0.0 | f1: 0.0,0.0,0.0,0.0
Los mejores parámetros para Neurological Disorders vs All son accuracy: 0.0,0.0,0.0,0.0 | f1: 0.0,0.0,0.0,0.0
Los mejores parámetros para Neurological Disorders vs Gastrointestinal diseases son accuracy: 0.0,0.0,0.0,0.0 | f1: 0.0,0.0,0.0,0.0
Los mejores parámetros para Neurological Disorders vs Metabolic Conditions son accuracy: 0.0

In [10]:
a[2]

,Healthy vs All,Healthy vs Neurological Disorders,Healthy vs Gastrointestinal diseases,Healthy vs Metabolic Conditions,Healthy vs Cardiovascular Diseases,Healthy vs Psychiatric Disorders,Neurological Disorders vs All,Neurological Disorders vs Gastrointestinal diseases,Neurological Disorders vs Metabolic Conditions,Neurological Disorders vs Cardiovascular Diseases,...,Gastrointestinal diseases vs All,Gastrointestinal diseases vs Metabolic Conditions,Gastrointestinal diseases vs Cardiovascular Diseases,Gastrointestinal diseases vs Psychiatric Disorders,Metabolic Conditions vs All,Metabolic Conditions vs Cardiovascular Diseases,Metabolic Conditions vs Psychiatric Disorders,Cardiovascular Diseases vs All,Cardiovascular Diseases vs Psychiatric Disorders,Psychiatric Disorders vs All
MMPD37460080ST,1.156355,-0.526941,1.142375,0.911382,0.375744,-1.042528,0.875089,NaN,NaN,NaN,...,-0.786362,NaN,NaN,NaN,-0.044544,NaN,NaN,0.087329,NaN,1.795172
MMPD56131850ST,1.605945,-0.988706,1.309607,0.386239,-0.113815,-1.050485,1.429566,NaN,NaN,NaN,...,-0.844416,NaN,NaN,NaN,0.490814,NaN,NaN,0.592361,NaN,1.583282
MMPD19623701ST,0.661603,-0.795355,0.611417,0.068728,0.288351,-0.835598,0.999836,NaN,NaN,NaN,...,-0.353440,NaN,NaN,NaN,0.484181,NaN,NaN,0.203292,NaN,1.293081
MMPD91893113ST,2.515486,-0.932836,1.381465,0.865445,0.088417,-0.815722,1.410675,NaN,NaN,NaN,...,-0.850995,NaN,NaN,NaN,0.170955,NaN,NaN,0.444019,NaN,1.447624
MMPD45969642ST,2.035862,-1.247850,1.332814,0.428889,0.384480,-0.894912,1.761898,NaN,NaN,NaN,...,-0.766452,NaN,NaN,NaN,0.466921,NaN,NaN,0.240692,NaN,1.504864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wHAXPI043592.8,2.488537,NaN,NaN,NaN,NaN,-1.425088,0.799234,NaN,NaN,NaN,...,-1.615645,NaN,NaN,-2.270109,-0.078509,NaN,-1.732383,0.218865,-1.384995,2.292847
wHAXPI043593.9,1.708568,NaN,NaN,NaN,NaN,-1.765451,1.751090,NaN,NaN,NaN,...,-1.429982,NaN,NaN,-1.909530,0.445347,NaN,-1.757549,0.892313,-1.208479,2.186763
wHAXPI043594.11,1.707576,NaN,NaN,NaN,NaN,-1.714599,1.075593,NaN,NaN,NaN,...,-1.091527,NaN,NaN,-2.198528,0.066584,NaN,-2.338601,0.800858,-1.350216,2.613241
wHAXPI047830.11,2.210000,NaN,NaN,NaN,NaN,-1.497183,1.069325,NaN,NaN,NaN,...,-1.036555,NaN,NaN,-1.702274,0.365191,NaN,-1.825345,1.182854,-1.071294,1.936437


In [8]:

def model_per_param(params):
    p1,p2,p3,p4=params[2:6]
    p1_p2=str(p1)+','+str(p2)
    p3_p4=str(p3)+','+str(p4)
    index_name=p1_p2+','+p3_p4
    
    df_metrics=params[-1]

    if df_metrics[df_metrics['p1,p2']==p1_p2].size!=0 and df_metrics[df_metrics['p3,p4']==p3_p4].size!=0:
        
        
        MH=df_metrics[df_metrics['p1,p2']==p1_p2]['MH_MN_GMHI'].values[0][0]
        MN=df_metrics[df_metrics['p3,p4']==p3_p4]['MH_MN_GMHI'].values[0][1]
        #####REVISAR
        GMHI=get_all_GMHI(params[1],MH,MN)
        metrics=get_accuracy(GMHI,params[0],params[6],params[7],params[8])
    else:
        df_metrics[df_metrics['p1,p2']==p1_p2].size==0 and df_metrics[df_metrics['p3,p4']==p3_p4].size==0
        #print(params[0],params[1],p1,p2,p3,p4,params[6],params[7],params[8])
        GMHI,MH,MN,metrics=train_GMHI(params[0],params[1],p1,p2,p3,p4,params[6],params[7],params[8])

    p1_p2=str(p1)+','+str(p2)
    p3_p4=str(p3)+','+str(p4)
    row=[p1_p2,p3_p4]
    
    row.extend([metrics[0],metrics[1],[MH,MN,GMHI]])
    
    df_metrics.loc[p1_p2+','+p3_p4,]=row
    return 
    


cols_and_index=np.append(meta[diagnosis_col_name].unique(),'All')
df_accuracy=pd.DataFrame(index=cols_and_index,columns=cols_and_index,data=np.nan)
df_f1=pd.DataFrame(index=cols_and_index,columns=cols_and_index,data=np.nan)

df_GMHI_accuracy=pd.DataFrame(index=tax.columns)
dict_best_sets_accuracy={}

df_GMHI_f1=pd.DataFrame(index=tax.columns)
dict_best_sets_f1={}



list_pairs=[]
for first_conditon in meta[diagnosis_col_name].unique():


    for second_condition in meta[diagnosis_col_name].unique():
        pair=set([first_conditon,second_condition])



        if pair in list_pairs:
            continue

        if first_conditon==second_condition:
            vs_name=first_conditon+' vs All'
            meta_first_and_second_condition=meta
            tax_first_and_second_condition=tax
            second_condition='All'
        else:
            vs_name=first_conditon+' vs '+second_condition
            meta_first_and_second_condition=meta[meta[diagnosis_col_name].isin([first_conditon,second_condition])]
            tax_first_and_second_condition=tax[meta_first_and_second_condition[sample_name]]
                
        list_pairs.append(pair)


        df_metrics=pd.DataFrame(columns=['p1,p2','p3,p4','accuracy','f1','MH_MN_GMHI'])
        
        param_grid = [[meta_first_and_second_condition,tax_first_and_second_condition,p1,p2,p3,p4,sample_name,first_conditon,diagnosis_col_name,df_metrics] 
                      for p1 in np.linspace(0.01, 0.7, num=1)
                      for p2 in np.linspace(0.01, 0.7, num=1)
                      for p3 in np.linspace(0.01, 0.7, num=1)
                      for p4 in np.linspace(0.01, 0.7, num=1)]

        
        for params in param_grid:
            model_per_param(params)



        best_param_for_accuracy=df_metrics.sort_values(by='accuracy')['accuracy'].index[-1]
        best_param_for_f1=df_metrics.sort_values(by='f1')['f1'].index[-1]
        print('los mejores parametros para '+vs_name+' son '+'accuracy: '+best_param_for_accuracy+' | f1: '+best_param_for_f1)


        #Mejores conjuntos MH y MN para este caso 
        best_GMHI_sets_for_accuracy=df_metrics.loc[best_param_for_accuracy,'MH_MN_GMHI'][:2]
        best_GMHI_sets_for_f1=df_metrics.loc[best_param_for_f1,'MH_MN_GMHI'][:2]

#RESULTADOS
         #Mejores scores
        df_accuracy.loc[first_conditon,second_condition]=df_metrics.loc[best_param_for_accuracy,'accuracy']
        df_f1.loc[first_conditon,second_condition]=df_metrics.loc[best_param_for_f1,'f1']

        df_accuracy.loc[second_condition,first_conditon]=df_accuracy.loc[first_conditon,second_condition]
        df_f1.loc[second_condition,first_conditon]=df_f1.loc[first_conditon,second_condition]

        #Mejores GMHI
        df_GMHI_f1[vs_name]=df_metrics.loc[best_param_for_accuracy,'MH_MN_GMHI'][2]
        df_GMHI_accuracy[vs_name]=df_metrics.loc[best_param_for_f1,'MH_MN_GMHI'][2]

        #Agregar sets a diccionarios, primero MH y MN
        dict_best_sets_accuracy[vs_name]=best_GMHI_sets_for_accuracy


        dict_best_sets_f1[vs_name]=best_GMHI_sets_for_f1


        #df_GMHI_accuracy[vs_name]=df_metrics.loc[best_param_for_accuracy,'MH_MN_GMHI'][2]
        #dict_best_sets_accuracy[vs_name]=df_metrics.loc[best_param_for_f1,'MH_MN_GMHI'][2]


los mejores parametros para Healthy vs All son accuracy: 0.01,0.01,0.01,0.01 | f1: 0.01,0.01,0.01,0.01
los mejores parametros para Healthy vs Neurological Disorders son accuracy: 0.01,0.01,0.01,0.01 | f1: 0.01,0.01,0.01,0.01
los mejores parametros para Healthy vs Gastrointestinal diseases son accuracy: 0.01,0.01,0.01,0.01 | f1: 0.01,0.01,0.01,0.01
los mejores parametros para Healthy vs Metabolic Conditions son accuracy: 0.01,0.01,0.01,0.01 | f1: 0.01,0.01,0.01,0.01
los mejores parametros para Healthy vs Cardiovascular Diseases son accuracy: 0.01,0.01,0.01,0.01 | f1: 0.01,0.01,0.01,0.01
los mejores parametros para Healthy vs Psychiatric Disorders son accuracy: 0.01,0.01,0.01,0.01 | f1: 0.01,0.01,0.01,0.01
los mejores parametros para Neurological Disorders vs All son accuracy: 0.01,0.01,0.01,0.01 | f1: 0.01,0.01,0.01,0.01
los mejores parametros para Neurological Disorders vs Gastrointestinal diseases son accuracy: 0.01,0.01,0.01,0.01 | f1: 0.01,0.01,0.01,0.01
los mejores parametros para 